In [2]:
from IPython.display import display, Markdown

import pandas as pd
import gc
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import RandomizedSearchCV, train_test_split, KFold, cross_val_score

from sklearn.metrics import roc_auc_score

from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, space_eval

from sklearn.tree import plot_tree

from sklearn.preprocessing import LabelEncoder, PolynomialFeatures, StandardScaler, OneHotEncoder

from scipy.sparse import hstack

# XGBoost
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt
import itertools
from scipy.stats import uniform
import pandas as pd # Para cargar los datos y hacer OHE.
import numpy as np  # Para lidiar con NaNs.
import time
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import ParameterSampler
from sklearn.metrics import confusion_matrix


#w2v
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import nltk
import re
import logging
import gensim
import numpy as np
import random
import os
import pickle
import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

random_state = 42
np.random.seed(random_state)


## Cargo los datos

In [4]:
# Load the competition data
comp_data = pd.read_csv("./competition_data.csv")

## Tuneo de datos

Retoque de fecha

In [5]:
comp_data['date'] = pd.to_datetime(comp_data['date'])
comp_data['month'] = comp_data['date'].dt.month
# comp_data['day'] = comp_data['date'].dt.day
comp_data['dayofweek'] = comp_data['date'].dt.dayofweek
# comp_data['hour'] = comp_data['date'].dt.hour


OHE en las categoricas

In [6]:
comp_data['is_pdp'].fillna(-1, inplace=True)
comp_data = pd.get_dummies(comp_data, columns=['is_pdp'], prefix='is_pdp')
comp_data = pd.get_dummies(comp_data, columns=['listing_type_id','logistic_type','platform','month','dayofweek']) #hour 

W2Vec en title

In [13]:

# Descarga de stopwords para español si no están descargadas ya
nltk.download('stopwords')

STOP_WORDS_SP = set(stopwords.words('spanish'))

def iterate_LN_corpus(path):
    """
    Genera un iterador para recorrer los archivos de texto en un directorio.

    Args:
        path (str): Ruta al directorio que contiene los archivos.

    Yields:
        str: Texto contenido en cada archivo.
    """
    articles = os.listdir(path)
    random.shuffle(articles)
    for art in articles:
        with open(path + art, encoding="utf-8") as f:
            raw_text = f.read()
        yield(raw_text)

def tokenizer(raw_text):
    """
    Tokeniza y preprocesa un texto.

    Args:
        raw_text (str): Texto sin procesar.

    Returns:
        list: Lista de oraciones, donde cada oración es una lista de palabras.
    """
    sentences = sent_tokenize(raw_text)
    sentences = [word_tokenize(e) for e in sentences]
    sentences = [[e2 for e2 in e1 if re.compile("[A-Za-z]").search(e2[0])] for e1 in sentences]
    sentences = [[e2.lower() for e2 in e1] for e1 in sentences]
    return(sentences)

def gen_sentences(path):
    """
    Genera una lista de oraciones a partir de archivos de texto en un directorio.

    Args:
        path (str): Ruta al directorio que contiene los archivos de texto.

    Returns:
        list: Lista de oraciones.
    """
    sentences = []
    n_arts = len(os.listdir(path))
    for i, art in tqdm.tqdm(enumerate(iterate_LN_corpus(path)), total=n_arts):
        sentences.extend(tokenizer(art))
    return(sentences)

def average_vectors(title_tokens, model, stopwords=None):
    """
    Calcula el vector promedio de un conjunto de tokens utilizando un modelo Word2Vec.

    Args:
        title_tokens (list): Lista de tokens.
        model (gensim.models.Word2Vec): Modelo Word2Vec.
        stopwords (set, optional): Conjunto de palabras stopwords. Defaults to None.

    Returns:
        numpy.ndarray: Vector promedio.
    """
    title_tokens = [e2 for e1 in title_tokens for e2 in e1]
    title_tokens = [e for e in title_tokens if e in model.wv]
    if stopwords is not None:
        title_tokens = [e for e in title_tokens if e not in stopwords]
    if len(title_tokens) == 0:
        output = np.zeros(model.wv.vector_size)
    else:
        output = np.array([model.wv.get_vector(e) for e in title_tokens]).mean(0)
    return output

def dummy_tokenizer(text_tokens):
    """
    Tokenizador dummy que simplemente devuelve los tokens de texto sin procesar.

    Args:
        text_tokens (list): Lista de tokens.

    Returns:
        list: Misma lista de tokens de entrada.
    """
    return text_tokens

# Configuración básica del logging
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s",
                    datefmt= '%H:%M:%S', level=logging.INFO)


#~ Análisis con datos de; TP ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

# Carga de datos desde un archivo CSV
# comp_data = pd.read_csv("competition_data.csv")
comp_data["title_tokens"] = comp_data["title"].map(tokenizer)

# Creación del modelo Word2Vec
w2v_tp = gensim.models.Word2Vec(vector_size=300,
                                window=3,
                                min_count=5,
                                negative=15,
                                sample=0.01,
                                workers=8,
                                sg=1)

# Creación del vocabulario a partir del corpus
w2v_tp.build_vocab([e2 for e1 in comp_data["title_tokens"].values for e2 in e1],
                   progress_per=10000)

# Entrenamiento del modelo Word2Vec
w2v_tp.train([e2 for e1 in comp_data["title_tokens"].values for e2 in e1],
             total_examples=w2v_tp.corpus_count,
             epochs=30, report_delay=1)

## Ejercicio de predicción con word2vec

# Obtención de embeddings de títulos utilizando el modelo Word2Vec
title_embs = comp_data["title_tokens"].map(lambda x: average_vectors(x, w2v_tp, STOP_WORDS_SP))
title_embs= np.array(title_embs.to_list())

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lucaspinera/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/lucaspinera/nltk_data'
    - '/Users/lucaspinera/opt/anaconda3/nltk_data'
    - '/Users/lucaspinera/opt/anaconda3/share/nltk_data'
    - '/Users/lucaspinera/opt/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [11]:
title_embs_df = pd.DataFrame(title_embs, columns=[f"emb_{i}" for i in range(title_embs.shape[1])])

# Concatenate comp_data with title_embs_df along the columns axis (axis=1)
comp_data = pd.concat([comp_data, title_embs_df], axis=1)


NameError: name 'title_embs' is not defined

In [7]:
comp_data.dtypes
# comp_data['emb_299']

accepts_mercadopago              bool
available_quantity              int64
avg_gmv_item_domain_30days    float64
avg_gmv_item_sel              float64
avg_gmv_seller_bday           float64
                               ...   
emb_295                       float64
emb_296                       float64
emb_297                       float64
emb_298                       float64
emb_299                       float64
Length: 379, dtype: object

Dropeo los que no ayudan al modelo

In [8]:
comp_data = comp_data.drop(
    columns=['accepts_mercadopago', 
            # 'available_quantity',
            # 'avg_gmv_item_domain_30days',
            # 'avg_gmv_item_sel',
            # 'avg_gmv_seller_bday',
            # 'avg_qty_orders_item_domain_30days',
            # 'avg_qty_orders_item_sel_30days',
            # 'avg_si_item_sel_30day',
            'benefit',              
            'boosted',
            'category_id',          
            # 'conversion',
            'date',             #chiche d efechas separar dia mes etc
            'deal_print_id',
            'domain_id',
            'etl_version',
            'free_shipping',    
            'fulfillment',
            'full_name',
            # 'health',
            # 'is_pdp',
            'product_id',
            'item_id',
            # 'listing_type_id', #hacer OHE
            # 'logistic_type',   #hacer OHE
            'main_picture',
            # 'offset',
            # 'original_price',
            # 'platform',        #hacer OHE
            # 'price',
            # 'print_position',
            'print_server_timestamp',
            # 'qty_items_dom',
            # 'qty_items_sel',
            'site_id',
            # 'sold_quantity',
            'tags',             #experimentar
            'title',            #W2vec
            # 'total_asp_item_domain_30days',
            # 'total_asp_item_sel_30days',
            # 'total_gmv_domain_bday',
            # 'total_gmv_item_30days',
            # 'total_items_domain',
            # 'total_items_seller',
            # 'total_orders_domain_30days',
            # 'total_orders_item_30days',
            # 'total_orders_sel_30days',
            # 'total_si_domain_30days',
            # 'total_si_item_30days',
            # 'total_si_sel_30days',
            # 'total_visits_domain',
            # 'total_visits_item',
            # 'total_visits_seller',
            'uid',
            'user_id',
            'warranty',
            # 'ROW_ID',
            'title_tokens'
            ]
)

## Validation Set - HOLDOUT SET

In [9]:

# Split into training and evaluation samples
train_data = comp_data[comp_data["ROW_ID"].isna()]
eval_data = comp_data[comp_data["ROW_ID"].notna()]
del comp_data
gc.collect()

y_train_all = train_data["conversion"]
X_train_all = train_data.drop(columns=["conversion", "ROW_ID"])
# X_train_all = X_train_all.select_dtypes(include='number') 
del train_data
gc.collect()

#hold out set
X_train, X_val, y_train, y_val = train_test_split(X_train_all, y_train_all, test_size=0.2, random_state=random_state)


In [10]:
X_train.dtypes

available_quantity                     int64
avg_gmv_item_domain_30days           float64
avg_gmv_item_sel                     float64
avg_gmv_seller_bday                  float64
avg_qty_orders_item_domain_30days    float64
                                      ...   
emb_295                              float64
emb_296                              float64
emb_297                              float64
emb_298                              float64
emb_299                              float64
Length: 355, dtype: object

## Modelo de aprendizaje - XGBoost

In [11]:
#ver uan forma de tener mejor parametos que gerar
params = {'max_depth': list(range(3, 12)),
          'learning_rate': uniform(scale = 0.03),
          'gamma': uniform(scale=15),               #chiche va de 1 a 20
          'reg_lambda': uniform(scale = 15),        # Parámetro de regularización.
          'subsample': uniform(0.5, 0.5),          # Entre 0.5 y 1.
          'min_child_weight': uniform(scale = 50),   #0 a 100
          'colsample_bytree': uniform(0.65, 0.35), # Entre 0.75 y 1.
          'n_estimators': list(range(500, 2500))    #ma que 500 tambein a la noche para que corra
         }

l_auc = []
l_grid = []
start = time.time()
best_score = 0
best_estimator = None
iterations = 300
for g in ParameterSampler(params, n_iter = iterations, random_state = random_state):
    clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic', seed = random_state, eval_metric = 'auc', **g, tree_method = 'gpu_hist') #enable_categorical = True
    clf_xgb.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = False)

    y_pred = clf_xgb.predict_proba(X_val)[:, 1] # Obtenemos la probabilidad de una de las clases (cualquiera).
    auc_roc = sklearn.metrics.roc_auc_score(y_val, y_pred)
    
    l_auc.append(auc_roc)
    l_grid.append(g)
    print(f'Roc-AUC actual; {auc_roc}')
    print(f'Con grid: {g}')
    # Guardamos si es mejor.
    if auc_roc > best_score:
        print('-'*10)
        print(f'Mejor valor de ROC-AUC encontrado: {auc_roc}')
        print(f'Grid{g}')
        print('-'*10)
        best_score = auc_roc
        best_grid = g
        best_estimator = clf_xgb

end = time.time()

df_aux = pd.DataFrame({'auc': l_auc, 'grid': l_grid})
df_aux.to_csv('grid_search.csv', index = False)

Roc-AUC actual; 0.8871928067599784
Con grid: {'colsample_bytree': 0.7810890415965769, 'gamma': 14.260714596148743, 'learning_rate': 0.02195981825434215, 'max_depth': 7, 'min_child_weight': 7.800932022121826, 'n_estimators': 966, 'reg_lambda': 1.4996237372700434, 'subsample': 0.7296244459829335}
----------
Mejor valor de ROC-AUC encontrado: 0.8871928067599784
Grid{'colsample_bytree': 0.7810890415965769, 'gamma': 14.260714596148743, 'learning_rate': 0.02195981825434215, 'max_depth': 7, 'min_child_weight': 7.800932022121826, 'n_estimators': 966, 'reg_lambda': 1.4996237372700434, 'subsample': 0.7296244459829335}
----------
Roc-AUC actual; 0.8962174210240451
Con grid: {'colsample_bytree': 0.7667980138986576, 'gamma': 2.143002268829112, 'learning_rate': 0.019526654188465584, 'max_depth': 7, 'min_child_weight': 48.49549260809972, 'n_estimators': 2015, 'reg_lambda': 14.078290635236252, 'subsample': 0.5003893829205072}
----------
Mejor valor de ROC-AUC encontrado: 0.8962174210240451
Grid{'colsa

me quedo con el mejor auc

In [12]:
print('ROC-AUC: %0.5f' % best_score)
print('Grilla:', best_grid)
print(f'Tiempo transcurrido: {str(end - start)} segundos')
print(f'Tiempo de entrenamiento por iteración: {str(round((end - start) / iterations, 2))} segundos')

ROC-AUC: 0.89728
Grilla: {'colsample_bytree': 0.6992999571329299, 'gamma': 2.6458047975933496, 'learning_rate': 0.014951033182184392, 'max_depth': 8, 'min_child_weight': 1.6939580123297493, 'n_estimators': 1684, 'reg_lambda': 5.435908486749497, 'subsample': 0.7902941751390218}
Tiempo transcurrido: 22536.81769824028 segundos
Tiempo de entrenamiento por iteración: 75.12 segundos


ahora con toda la data el prosible auc:

In [13]:
y_pred = best_estimator.predict_proba(X_val)[:, 1]
auc_roc = sklearn.metrics.roc_auc_score(y_val, y_pred)
print('AUC-ROC test: %0.5f' % auc_roc)

AUC-ROC test: 0.89728


entreno con toda la data train+val

In [14]:
# best_grid = {'colsample_bytree': 0.7860370513913187, 'gamma': 4.070235476608438, 'learning_rate': 0.02486212527455788, 'max_depth': 11, 'min_child_weight': 14.046725484369038, 'n_estimators': 879, 'reg_lambda': 4.1399877303381505, 'subsample': 0.6481367528520412}

In [15]:
clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic', seed = random_state, eval_metric = 'auc', **best_grid,tree_method = 'gpu_hist') #enable_categorical = True
clf_xgb.fit(X_train_all, y_train_all, verbose = False)
final_model = clf_xgb

In [ ]:

y_pred_submi = final_model.predict_proba(eval_data.drop(columns=["conversion", "ROW_ID"]))[:, 1]

## Submission

In [ ]:
# y_preds = cls.predict_proba(eval_data.drop(columns=["ROW_ID"]))[:, cls.classes_ == 1].squeeze()
# Make the submission file
submission_df = pd.DataFrame({"ROW_ID": eval_data["ROW_ID"], "conversion": y_pred_submi})
submission_df["ROW_ID"] = submission_df["ROW_ID"].astype(int)
submission_df.to_csv("mimir.csv", sep=",", index=False)